In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import itertools

In [2]:
GOLD = pd.read_csv('data/macro_data/XAUUSD_mt5.csv', index_col=0, parse_dates=['date'])
GOLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6491 entries, 0 to 6490
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         6491 non-null   datetime64[ns]
 1   open         6491 non-null   float64       
 2   high         6491 non-null   float64       
 3   low          6491 non-null   float64       
 4   close        6491 non-null   float64       
 5   tick_volume  6491 non-null   int64         
 6   spread       6491 non-null   int64         
 7   real_volume  6491 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 456.4 KB


In [3]:
GOLD.head()

,date,open,high,low,close,tick_volume,spread,real_volume
0,1998-04-22,310.30,314.40,310.0,313.5,561,0,0
1,1998-04-23,312.60,315.20,311.9,314.3,491,0,0
2,1998-04-24,313.40,314.20,311.8,312.9,431,0,0
3,1998-04-27,312.25,313.65,308.9,309.9,716,0,0
4,1998-04-28,309.39,310.90,306.3,308.2,802,0,0


In [4]:
GOLD['month'] = GOLD['date'].dt.month
GOLD['year'] = GOLD['date'].dt.year

In [5]:
GOLD.head()

,date,open,high,low,close,tick_volume,spread,real_volume,month,year
0,1998-04-22,310.30,314.40,310.0,313.5,561,0,0,4,1998
1,1998-04-23,312.60,315.20,311.9,314.3,491,0,0,4,1998
2,1998-04-24,313.40,314.20,311.8,312.9,431,0,0,4,1998
3,1998-04-27,312.25,313.65,308.9,309.9,716,0,0,4,1998
4,1998-04-28,309.39,310.90,306.3,308.2,802,0,0,4,1998


In [6]:
CPI = pd.read_csv('data/macro_data/CPI_previous_period.csv', parse_dates=['DATE'])
CPI.head()

,DATE,CPI_previous_period
0,1987-01-11,0.086730
1,1987-01-12,0.000000
2,1988-01-01,0.259965
3,1988-01-02,0.259291
4,1988-01-03,0.431034


In [7]:
CPI['DATE'] = CPI['DATE'].dt.strftime('%Y-%m-%d')
CPI['DATE'] = pd.to_datetime(CPI['DATE'], format='%Y-%d-%m')

CPI['month'] = CPI['DATE'].dt.month
CPI['year'] = CPI['DATE'].dt.year

In [8]:
CPI.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE                 417 non-null    datetime64[ns]
 1   CPI_previous_period  417 non-null    float64       
 2   month                417 non-null    int64         
 3   year                 417 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 13.2 KB


In [9]:
CPI.tail()

,DATE,CPI_previous_period,month,year
412,2022-03-01,1.335138,3,2022
413,2022-04-01,0.558253,4,2022
414,2022-05-01,1.102352,5,2022
415,2022-06-01,1.373608,6,2022
416,2022-07-01,-0.011812,7,2022


In [16]:

GOLD_CPI = []
for i in range(len(CPI)):
    
    # slices the dataframe based on months and year from the macro exonomic dataset:
    # eg: CPI , interest rate dataset
    df_subset_yearly = GOLD.loc[((GOLD['year'] == CPI['year'][i])) & ((GOLD['month'] == CPI['month'][i]))]

    # checls with the condition and fills in the macor econoic data for daily frequency 
    if [((df_subset_yearly['year'] == CPI['year'][i])) & ((df_subset_yearly['month'] == CPI['month'][i]))]:
        df_subset_yearly['CPI'] = CPI['CPI_previous_period'][i]
        
    else:
        continue
    GOLD_CPI.append(df_subset_yearly)
GOLD_CPI = pd.concat(GOLD_CPI)

print(GOLD_CPI)

In [14]:
GOLD_CPI.tail(100)

,date,open,high,low,close,tick_volume,spread,real_volume,month,year,CPI
6336,2022-03-11,1996.70,1999.12,1958.70,1987.31,84636,0,0,3,2022,1.335138
6337,2022-03-14,1982.16,1984.08,1949.66,1950.72,83698,0,0,3,2022,1.335138
6338,2022-03-15,1952.11,1954.60,1907.09,1917.69,90366,0,0,3,2022,1.335138
6339,2022-03-16,1918.94,1929.48,1895.08,1926.11,83374,0,0,3,2022,1.335138
6340,2022-03-17,1926.56,1949.81,1923.19,1942.49,73924,0,0,3,2022,1.335138
...,...,...,...,...,...,...,...,...,...,...,...
6431,2022-07-25,1726.70,1736.22,1714.74,1719.64,77653,0,0,7,2022,-0.011812
6432,2022-07-26,1719.10,1728.16,1713.62,1717.20,76426,0,0,7,2022,-0.011812
6433,2022-07-27,1717.39,1740.24,1711.50,1734.33,81487,0,0,7,2022,-0.011812
6434,2022-07-28,1734.65,1757.10,1734.07,1755.83,96431,0,0,7,2022,-0.011812


In [15]:
#GOLD_CPI.to_csv('data/macro_data/GOLD_CPI.csv')